In [ ]:
import sys

sys.path.append("../..")
from dotenv import load_dotenv

load_dotenv()
from llama_index.agent import OpenAIAgent
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.callbacks import CallbackManager
from llama_index.schema import Document
from llama_index.indices import VectorStoreIndex
from llama_index.callbacks.async_iter import AsyncIteratorCallbackHandler

ignore_events = []
stream = AsyncIteratorCallbackHandler(
    event_starts_to_ignore=ignore_events,
    event_ends_to_ignore=ignore_events,
    verbose=False,
)

callback_manager = CallbackManager(
    handlers=[stream],
)

index = VectorStoreIndex.from_documents(
    [
        Document(
            content="I went to the shops today.",
            metadata={"date": "2021-01-01"},
        ),
    ],
    show_progress=True,
)

tool = QueryEngineTool(
    query_engine=index.as_query_engine(),
    metadata=ToolMetadata(
        name="my_diary",
        description="A tool for looking at my diary.",
    ),
)
agent = OpenAIAgent.from_tools([tool], callback_manager=callback_manager, verbose=True)

/Users/jordanparker/.pyenv/versions/3.9.13/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.68it/s]


In [ ]:
agent.chat("What did I do a week ago?")

{'event': 'agent_step.start', 'event_id': 'ab8644d2-1004-4c42-a0fb-5529d14b07fa', 'parent_id': 'root'}
{"event": "agent_step.start", "event_id": "ab8644d2-1004-4c42-a0fb-5529d14b07fa", "parent_id": "root"}
STARTING TURN 1
---------------

{'event': 'llm.start', 'event_id': 'f9bf90ca-78d2-4914-948c-86ee9ffe8000', 'parent_id': 'ab8644d2-1004-4c42-a0fb-5529d14b07fa'}
{"event": "llm.start", "event_id": "f9bf90ca-78d2-4914-948c-86ee9ffe8000", "parent_id": "ab8644d2-1004-4c42-a0fb-5529d14b07fa"}
{'event': 'llm.end', 'event_id': 'f9bf90ca-78d2-4914-948c-86ee9ffe8000', 'data': {<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What did I do a week ago?', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='What did I do a week ago?', additional_kwargs={})], <EventPayload.RESPONSE: 'response'>: ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [ChatCompletio

TypeError: Object of type ChatMessage is not JSON serializable

In [ ]:
async for event in stream.aiter():
    print(event)